In [2]:
import warnings
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import iqr
import matplotlib.backends.backend_pdf

warnings.filterwarnings("ignore")


In [3]:
matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
matplotlib.style.use('ggplot')

# Create models from data
def best_fit_distribution(data, bins, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    DISTRIBUTIONS = [st.alpha, st.beta,st.chi,st.chi2, st.dgamma,st.dweibull,
                     st.erlang,st.expon,st.exponnorm,st.exponweib,st.gennorm,st.gamma,
                     st.invgamma,st.invgauss,st.invweibull,st.logistic,st.loggamma,st.lognorm,
                     st.maxwell,st.norm,st.powernorm, st.triang,st.uniform]

    # Best Parameters
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    # Estimate distribution parameters
    for distribution in DISTRIBUTIONS:

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                    end
                except Exception:
                    pass

                # check if distribution is best
                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse

        except Exception:
            print('No fit!')
            pass

    return (best_distribution.name, best_params)

def generate_pdf(dist, params, size=1000):
    """ Generate PDF """

    # Separate parts of parameters
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]

    # Get start and end points of distribution
    start = dist.ppf(0.01, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.01, loc=loc, scale=scale)
    end = dist.ppf(0.99, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.99, loc=loc, scale=scale)

    # Store PDF values
    x = np.linspace(start, end, size)
    y = dist.pdf(x, loc=loc, scale=scale, *arg)
    pdf = pd.Series(y, x)

    return pdf

In [4]:
# Create empty df
df_dist = pd.DataFrame(columns=['GroupedData', 'Distribution'])

In [6]:
# Read distribution
data_set = pd.read_csv("../Data/Packaging.csv")

In [ ]:
# Set output
out_pdf = r'C:\Users\batuhan.organ\Desktop\IEOzU\packaging.pdf'

pdf_write = matplotlib.backends.backend_pdf.PdfPages(out_pdf)

for data_name, sample_data in data_set.groupby(['S', 'F', 'P']):
    
    data = sample_data['Processing_Rate']

     # Determine num of bins by Freedman-Diaconis Rule
    y_arr = np.array(data)
    y_iqr = iqr(y_arr, rng=(25,75), interpolation='midpoint')
    h = 2 * y_iqr * len(data)**(-1/3)
    num_of_bins = int(round((max(data) - min(data)) / h,0))
    
    # Plot Hist
    figs = plt.figure()
    fig = plt.figure(figsize=(12,8))
    ax = data.plot(kind='hist', bins=num_of_bins, density=True, alpha=0.5)
    # Save plot limits
    dataYLim = ax.get_ylim()


    # Find best fit distribution
    best_fit_name, best_fit_params = best_fit_distribution(data, num_of_bins, ax)
    best_dist = getattr(st, best_fit_name)
    
    # Update plot titles
    ax.set_ylim(dataYLim)
    ax.set_title(u'Process Rate.\n All Fitted Distributions \n' + str(data_name))
    ax.set_xlabel(u'Process Rate (Pounds per Hour)')
    ax.set_ylabel('Frequency')

    pdf_write.savefig(fig)
    matplotlib.pyplot.close(fig)
    
    # Make PDF with best params 
    pdf = generate_pdf(best_dist, best_fit_params)

    # Display
    fig = plt.figure(figsize=(12,8))
    ax = pdf.plot(lw=2, label='PDF', legend=True)
    data.plot(kind='hist', bins=50, density=True, alpha=0.5, label='Data', legend=True, ax=ax)

    # Set grouped data label
    param_names = (best_dist.shapes + ', loc, scale').split(', ') if best_dist.shapes else ['loc', 'scale']
    param_str = ', '.join(['{}={:0.2f}'.format(k,v) for k,v in zip(param_names, best_fit_params)])
    dist_str = '{}({})'.format(best_fit_name, param_str)
    
    # Append rows in Empty Dataframe by adding dictionaries
    df_dist = df_dist.append({'GroupedData': str(data_name), 'Distribution': dist_str}, ignore_index=True)

    dist_str = dist_str + '\n '+ str(data_name)
    
    
    ax.set_title(u'Packaging Process Rate. with best fit distribution\n' + dist_str)
    ax.set_xlabel(u'Process Rate (Pounds per Hour)')
    ax.set_ylabel('Frequency')
    pdf_write.savefig(fig)
    matplotlib.pyplot.close(fig)
pdf_write.close()

In [ ]:
# Save output
df_dist.to_csv("data_with_distributions.csv", index=False)